In [1]:
!pip install tensorflow --upgrade --force-reinstall
!pip install tf-nightly-gpu
import tensorflow as tf
import numpy as np
import os

     |████████████████████████████████| 320.4MB 48kB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 133kB 48.8MB/s 
     |████████████████████████████████| 1.3MB 43.7MB/s 
     |████████████████████████████████| 3.4MB 40.9MB/s 
     |████████████████████████████████| 460kB 34.9MB/s 
     |████████████████████████████████| 26.1MB 1.5MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 6.8MB 41.6MB/s 
     |████████████████████████████████| 2.9MB 38.9MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 788kB 41.2MB/s 
     |████████████████████████████████| 92kB 8.1MB/s 
     |████████████████████████████████| 788kB 24.8MB/s 
     |████████████████████████████████| 307kB 48.9MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
   

     |████████████████████████████████| 341.4MB 50kB/s 
     |████████████████████████████████| 460kB 31.9MB/s 
     |████████████████████████████████| 9.2MB 43.2MB/s 


In [2]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="flower_photos.tgz", 
                                   extract=True, cache_subdir='/content',)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

228818944/228813984 [==============================] - 3s 0us/step


In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

Found 2939 images belonging to 5 classes.


In [4]:
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')


Found 731 images belonging to 5 classes.


In [5]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 5))

In [6]:
print(train_generator.class_indices)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [8]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

In [9]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')


9412608/9406464 [==============================] - 0s 0us/step


In [10]:
base_model.trainable = False

In [12]:
model = tf.keras.Sequential([
      base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
epochs = 10

history = model.fit(train_generator, 
                    epochs=epochs, 
                    validation_data=val_generator)

Epoch 1/10
46/46 [==============================] - 150s 3s/step - loss: 1.6037 - accuracy: 0.5440 - val_loss: 0.6786 - val_accuracy: 0.7209
Epoch 2/10
46/46 [==============================] - 146s 3s/step - loss: 0.5480 - accuracy: 0.7514 - val_loss: 0.5371 - val_accuracy: 0.8140
Epoch 3/10
46/46 [==============================] - 145s 3s/step - loss: 0.3228 - accuracy: 0.8904 - val_loss: 0.4168 - val_accuracy: 0.8495
Epoch 4/10
46/46 [==============================] - 145s 3s/step - loss: 0.1937 - accuracy: 0.9423 - val_loss: 0.4919 - val_accuracy: 0.8440
Epoch 5/10
46/46 [==============================] - 145s 3s/step - loss: 0.1257 - accuracy: 0.9664 - val_loss: 0.4388 - val_accuracy: 0.8523
Epoch 6/10
46/46 [==============================] - 144s 3s/step - loss: 0.0960 - accuracy: 0.9723 - val_loss: 0.4509 - val_accuracy: 0.8591
Epoch 7/10
46/46 [==============================] - 145s 3s/step - loss: 0.0601 - accuracy: 0.9881 - val_loss: 0.4610 - val_accuracy: 0.8509
Epoch 8/10
46

In [15]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: assets


In [16]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [17]:
from google.colab import files
files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>